# Watched Dictionaries

---

keeping track of key-value changes in a dict

In [1]:
from spectate import watched_type, Bunch

We use `watched_type` to create an `EventfulDict` that enables callbacks before and after an element of the dictionary is set. The funtion's signature requries the name of the new eventful type, a base class from which that new type should inherit, and the series of attributes on that base class which can register a callback. In this case we enable callbacks for methods which can set or delete items:

+ `__setitem__`
+ `__delitem__`
+ `setdefault`
+ `popitem`.
+ `pop`
+ `update`


In [2]:
EventfulDict = watched_type('EventfulList', dict, '__setitem__',
    '__delitem__', 'setdefault', 'update', 'pop', 'popitem')

In [3]:
# a sentinel signifying an empty element
class Empty(object):
    def __repr__(self): return "Empty"
Empty = Empty()

Beforebacks
-----------

+ Have a signature of ``(instance, call)``

    + ``instance`` is the owner of the method
    + ``call`` is a ``Bunch`` with the keys:

        + ``'name'`` - the name of the method which was called
        + ``'args'`` - the arguments which that method will call
        + ``'kwargs'`` - the keywords which that method will call

+ Can ``return`` a value which gets passed on to its respective afterback.
+ If an error is encountered, the wrapper will ``return`` the original ``call``
as if it were that of the beforeback, and set ``error`` in the ``answer``
bunch of its afterback. This way, if the base method call is valid, it's not
obstructed by a raised beforeback. Thus, beforeback errors should be handled
in an afterback.

Afterbacks
----------

+ Have a signature of ``(instance, answer)``

    + ``instance`` is the owner of the method
    + ``answer`` is a ``Bunch`` with the keys:

        + ``'name'`` - the name of the method which was called
        + ``'value'`` - the value returned by the method
        + ``'before'`` - the value returned by the respective beforeback
        + ``'error'`` - None, or the error encountered in the beforeback

+ Responcible for raising beforeback errors.
+ Should not ``return``

In [4]:
# beforebacks

def will_update(inst, call):
    return {k: inst.get(k, Empty) for k, v
        in dict(call.args[0]).items()}

def will_change_item(inst, call):
    return call.args[0], inst.get(call.args[0], Empty)

def will_pop_item(inst, call):
    k = inst.keys()[0]
    return k, inst[k]

# afterbacks

def did_update(inst, answer):
    for k, v in answer.before.items():
        did_change_item(inst, Bunch(before=(k, v)))

def did_change_item(inst, answer):
    key, old = answer.before
    new = inst.get(key, Empty)
    if new != old:
        print("{%s: %s} -> {%s: %s}" %
            (key, old, key, new))

Instances of `EventfulDict` have a public method `spectator_callback` which is used to store unique callback pairs. 

Each pair can have a beforeback, and/or an afterback which are specified in the method signature with the keywords `before` and `after`.

In [5]:
edict = EventfulDict({'a': 1, 'b':2})

edict.spectator_callback(
    '__setitem__', '__delitem__',
    'setdefault', 'pop',
    before=will_change_item,
    after=did_change_item)

edict.spectator_callback('popitem',
    before=will_pop_item,
    after=did_change_item)

edict.spectator_callback('update',
    before=will_update,
    after=did_update)

In [6]:
# tests
edict['a'] = 2
edict.pop('a')
edict.popitem()
edict.setdefault('c', 3)
edict.update({'a':1, 'b':2})
del edict['c']
# won't notify
edict['a'] = 1

{a: 1} -> {a: 2}
{a: 2} -> {a: Empty}
{b: 2} -> {b: Empty}
{c: Empty} -> {c: 3}
{a: Empty} -> {a: 1}
{b: Empty} -> {b: 2}
{c: 3} -> {c: Empty}


In [7]:
edict

{'a': 1, 'b': 2}